# Tutorial
OceanSpy builds on software packages developed by the [Pangeo](http://pangeo-data.org/) community.  
OceanSpy is documented [here](https://oceanspy.readthedocs.io/en/latest/), and makes heavy use of [xarray](http://xarray.pydata.org/en/stable/index.html) and [xgcm](http://xgcm.readthedocs.io/en/latest/).
Familiarity with these packages is recommended, but not required.

Install and import OceanSpy
-------------------------------

### Install
Run the following commands only if OceanSpy is not installed yet.

In [ ]:
import sys
!conda install --yes --prefix {sys.prefix} dask distributed bottleneck netCDF4
!conda install --yes --prefix {sys.prefix} -c conda-forge xarray cartopy esmpy 
!conda install --yes --prefix {sys.prefix} -c pyviz hvplot geoviews
!{sys.executable} -m pip install geopy xgcm xesmf oceanspy

###  Import
OceanSpy is ``Tab``-friendly: every OceanSpy component (objects, modules, functions, ...) is documented and self-explanatory names are used. Check the [API](https://oceanspy.readthedocs.io/en/latest/installation.html) section, use ``help()``, and take advantage of the ``Tab`` key. Most of the functions demonstrated here have additional options for expert users!

In [1]:
import oceanspy as ospy
help(ospy)

Help on package oceanspy:

NAME
    oceanspy

PACKAGE CONTENTS
    compute
    open_dataset
    subsample
    utils
    visualize

DATA
    __email__ = 'mattia.almansi@jhu.edu'

VERSION
    0.0.10

AUTHOR
    Mattia Almansi

FILE
    /home/idies/workspace/Temporary/malmans2/scratch/testOspy/oceanspy/oceanspy/__init__.py




Open dataset
--------------
OceanSpy will create two crucial variables: ``ds`` is a ``xarray.Dataset`` containing model outputs, while ``info`` is an object containing several information used by OceanSpy (e.g., ``info.grid`` is a ``xgcm.Grid`` object). If you want to learn more about ``Dataset`` and ``Grid``, take a look at [xarray](http://xarray.pydata.org/en/stable/index.html) and [xgcm](http://xgcm.readthedocs.io/en/latest/) documentations, examples, and tutorials.  
NOTE: it might take a couple fo minutes when you open a dataset for the very first time.

In [2]:
ds, info = ospy.open_dataset.exp_ASR()
print('\n\nThis is a dataset:\n',ds)
print('\n\nThis class contains information that will be used by OceanSpy\n',info)

Opening exp_ASR


This is a dataset:
 <xarray.Dataset>
Dimensions:     (X: 960, Xp1: 961, Y: 880, Yp1: 881, Z: 216, Zl: 216, Zp1: 217, Zu: 216, time: 1464)
Coordinates:
  * Z           (Z) float64 -1.0 -3.5 -7.0 -11.5 -17.0 -23.5 -31.0 -39.5 ...
  * Zp1         (Zp1) float64 0.0 -2.0 -5.0 -9.0 -14.0 -20.0 -27.0 -35.0 ...
  * Zu          (Zu) float64 -2.0 -5.0 -9.0 -14.0 -20.0 -27.0 -35.0 -44.0 ...
  * Zl          (Zl) float64 0.0 -2.0 -5.0 -9.0 -14.0 -20.0 -27.0 -35.0 ...
  * X           (X) float64 -46.92 -46.83 -46.74 -46.65 -46.57 -46.48 -46.4 ...
  * Y           (Y) float64 56.81 56.85 56.89 56.93 56.96 57.0 57.04 57.08 ...
  * Xp1         (Xp1) float64 -46.96 -46.87 -46.78 -46.7 -46.61 -46.53 ...
  * Yp1         (Yp1) float64 56.79 56.83 56.87 56.91 56.95 56.98 57.02 ...
  * time        (time) datetime64[ns] 2007-09-01 2007-09-01T06:00:00 ...
Data variables:
    drC         (Zp1) float64 -1.0 -2.5 -3.5 -4.5 -5.5 -6.5 -7.5 -8.5 -9.5 ...
    drF         (Z) float64 -2.0 -3.0 -4.0 -5

Visualize
----------
Run ``ospy.visualize.interactive()`` to plot ``ds``. OceanSpy will print several information, and a GUI allows to change some parameters, such as time and vertical levels. You can also zoom in, and save any plots.  
NOTE: The whole dataset is big and OceanSpy's response might be slow (especially 3D variables).

In [3]:
_ = ospy.visualize.interactive(ds, info)

description : r cell center separation

Dimension : Zp1
      units : meters
      long_name : vertical coordinate of cell interface
      positive : up
      axis : Z
      c_grid_axis_shift : -0.5


:Curve   [Zp1]   (drC)

![alt](images/tutorial_ds.png)

You can produce customized plot using ``hvplot_kwargs``, or you can make your own plots from scratch using [xarray plot()](http://xarray.pydata.org/en/stable/plotting.html), [matplotlib](https://matplotlib.org/), and [cartopy](https://scitools.org.uk/cartopy/docs/latest/).

In [4]:
help(ospy.visualize.interactive)

Help on function interactive in module oceanspy.visualize:

interactive(dsORda, info, hvplot_kwargs={})
    GUI for plotting data interactively using hvPlot.
    Plot maps using Orthographic projection, 
    and recognize vertical sections/profiles
    
    Parameters
    ----------
    dsORda: xarray.Dataset or xarray.DataArray
    info: oceanspy.open_dataset._info
    hvplot_kwargs: dict
        Keyword arguments for hvPlot: https://hvplot.pyviz.org/user_guide/index.html
    
    Returns
    -------
    GUI



Subsample
----------

### Cutout
It is better to work with subsample when possible.
Run ``ospy.subsample.cutout()`` to reduce the dataset. Check the documentation to learn more about available options.  
NOTE: most of the functions affect the original ``ds`` and ``info``. If you are planning to use the input variables in the future, set ``deep_copy=True`` (it takes a few seconds), otherwise use ``deep_copy=False`` (default) to save time. Check out the differences between the original dataset and the cutout (e.g., print and plot ``ds`` and ``ds_cut``).  

In [5]:
ds_cut, info_cut = ospy.subsample.cutout(ds,
                                         info,
                                         varList    = ['Eta', 'Depth'],
                                         latRange   = [60,  71],
                                         lonRange   = [-40, -10],
                                         depthRange = [0, -100],
                                         timeRange  = ['2007-09-01T00', '2007-09-30T18'],
                                         timeFreq   = '1D',
                                         sampMethod = 'mean',
                                         deep_copy  = True)
ds_cut

Copying ds and info
Cutting out
Resampling timeseries


<xarray.Dataset>
Dimensions:  (X: 677, Xp1: 678, Y: 575, Yp1: 576, Z: 12, Zl: 12, Zp1: 13, Zu: 12, time: 30)
Coordinates:
  * X        (X) float64 -39.97 -39.92 -39.88 -39.83 -39.79 -39.75 -39.7 ...
  * Y        (Y) float64 60.01 60.03 60.05 60.07 60.09 60.11 60.13 60.14 ...
  * Z        (Z) float64 -1.0 -3.5 -7.0 -11.5 -17.0 -23.5 -31.0 -39.5 -49.0 ...
  * Zp1      (Zp1) float64 0.0 -2.0 -5.0 -9.0 -14.0 -20.0 -27.0 -35.0 -44.0 ...
  * Zu       (Zu) float64 -2.0 -5.0 -9.0 -14.0 -20.0 -27.0 -35.0 -44.0 -54.0 ...
  * Zl       (Zl) float64 0.0 -2.0 -5.0 -9.0 -14.0 -20.0 -27.0 -35.0 -44.0 ...
  * Xp1      (Xp1) float64 -39.99 -39.94 -39.9 -39.86 -39.81 -39.77 -39.72 ...
  * Yp1      (Yp1) float64 60.0 60.02 60.04 60.06 60.08 60.1 60.12 60.14 ...
  * time     (time) datetime64[ns] 2007-09-01 2007-09-02 2007-09-03 ...
Data variables:
    Depth    (Y, X) float64 2.514e+03 2.519e+03 2.532e+03 2.546e+03 ...
    Eta      (time, Y, X) float64 dask.array<shape=(30, 575, 677), chunksize=(1, 575, 67

### Save

``ds`` and ``info`` can be saved at any time (NetCDF format for ``ds``, object format for ``info``) to create checkpoints, or to download data. For example, use ``ds_cut`` and ``info_cut`` to create a checkpoint. This cutout is quite small, and you can store it in your ``Storage`` directory. Take a look at this [Important Information about Compute Container File Storage](https://apps.sciserver.org/compute/).

In [6]:
path = './tmp/checkpoint'
ospy.utils.save_ds_info(ds_cut, info_cut, path)
ds_cut, info_cut = ospy.utils.open_ds_info(path)

Saving ds to ./tmp/checkpoint.nc


/home/idies/miniconda3/lib/python3.6/site-packages/dask/array/numpy_compat.py:28: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


Saving info to ./tmp/checkpoint.obj
Opening ds from ./tmp/checkpoint.nc
Opening info from ./tmp/checkpoint.obj


Compute
--------
``ospy.compute`` includes several functions and algorithms that add new variables to the dataset. ``ds`` and ``info`` are always input and output of these function, and the ``deep_copy`` option is always available. Here is the list of functions, and their descriptions:

In [7]:
help(ospy.compute)

Help on module oceanspy.compute in oceanspy:

NAME
    oceanspy.compute - Compute: add new variables to the dataset

FUNCTIONS
    EKE(ds, info, deep_copy=False)
        Compute Eddy Kinetic Energy and add to dataset.
        http://mitgcm.readthedocs.io/en/latest/algorithm/algorithm.html#kinetic-energy
        Note: non-hydrostatic term is omitted (\epsilon_{nh}=0)
        
        Parameters
        ----------
        ds: xarray.Dataset
        info: oceanspy.open_dataset._info
        deep_copy: bool
            If True, deep copy ds and infod
        
        Returns
        -------
        ds: xarray.Dataset 
        info: oceanspy.open_dataset._info
    
    Ertel_PV(ds, info, deep_copy=False)
        Compute Ertel Potential Vorticity and add to dataset.
        Eq. 2.2 in OC3D, Klinger and Haine, 2018.
        
        Parameters
        ----------
        ds: xarray.Dataset
        info: oceanspy.open_dataset._info
        deep_copy: bool
            If True, deep copy ds and i

### Potential Density Anomaly

To test ``ospy.compute``, create a new cutout with all the original variables, then compute the potential density anomamly, and explore the new dataset:

In [8]:
ds_cut, info_cut = ospy.subsample.cutout(ds,
                                         info,
                                         latRange   = [60,  71],
                                         lonRange   = [-40, -10],
                                         depthRange = [0, -100],
                                         timeRange  = ['2007-09-01T00', '2007-09-30T18'],
                                         timeFreq   = '1D',
                                         sampMethod = 'mean',
                                         deep_copy  = True)
ds_Sigma0, info_Sigma0 = ospy.compute.Sigma0(ds_cut, info_cut)
ds_Sigma0

Copying ds and info
Cutting out
Resampling timeseries
Computing Sigma0


<xarray.Dataset>
Dimensions:     (X: 677, Xp1: 678, Y: 575, Yp1: 576, Z: 12, Zl: 12, Zp1: 13, Zu: 12, time: 30)
Coordinates:
  * X           (X) float64 -39.97 -39.92 -39.88 -39.83 -39.79 -39.75 -39.7 ...
  * Xp1         (Xp1) float64 -39.99 -39.94 -39.9 -39.86 -39.81 -39.77 ...
  * Y           (Y) float64 60.01 60.03 60.05 60.07 60.09 60.11 60.13 60.14 ...
  * Yp1         (Yp1) float64 60.0 60.02 60.04 60.06 60.08 60.1 60.12 60.14 ...
  * Z           (Z) float64 -1.0 -3.5 -7.0 -11.5 -17.0 -23.5 -31.0 -39.5 ...
  * Zl          (Zl) float64 0.0 -2.0 -5.0 -9.0 -14.0 -20.0 -27.0 -35.0 ...
  * Zp1         (Zp1) float64 0.0 -2.0 -5.0 -9.0 -14.0 -20.0 -27.0 -35.0 ...
  * Zu          (Zu) float64 -2.0 -5.0 -9.0 -14.0 -20.0 -27.0 -35.0 -44.0 ...
  * time        (time) datetime64[ns] 2007-09-01 2007-09-02 2007-09-03 ...
Data variables:
    drC         (Zp1) float64 -1.0 -2.5 -3.5 -4.5 -5.5 -6.5 -7.5 -8.5 -9.5 ...
    drF         (Z) float64 -2.0 -3.0 -4.0 -5.0 -6.0 -7.0 -8.0 -9.0 -10.0 ...
    

### Ertel PV

Some of the diagnostics produced by OceanSpy make use of several ``ospy.compute`` functions. For example, Ertel Potential Vorticity is computed using the following equation:   
``Ertel_PV`` $=(f+\zeta)\frac{N^2}{g}+\frac{(\boldsymbol{\zeta}_h+e\hat{\boldsymbol{y}})\cdot\boldsymbol{\nabla}_h\rho}{\rho_0}$.   
All the variables computed by OceanSpy that show up in the ``Ertel_PV`` equation are also added to ``ds``.

In [9]:
ds_PV, info_PV = ospy.compute.Ertel_PV(ds_cut, info_cut, deep_copy  = True)
ds_PV

Copying ds and info
Computing N2
Computing momVort1
Computing momVort2
Computing Ertel_PV


<xarray.Dataset>
Dimensions:     (X: 677, Xp1: 678, Y: 575, Yp1: 576, Z: 12, Zl: 12, Zp1: 13, Zu: 12, time: 30)
Coordinates:
  * X           (X) float64 -39.97 -39.92 -39.88 -39.83 -39.79 -39.75 -39.7 ...
  * Xp1         (Xp1) float64 -39.99 -39.94 -39.9 -39.86 -39.81 -39.77 ...
  * Y           (Y) float64 60.01 60.03 60.05 60.07 60.09 60.11 60.13 60.14 ...
  * Yp1         (Yp1) float64 60.0 60.02 60.04 60.06 60.08 60.1 60.12 60.14 ...
  * Z           (Z) float64 -1.0 -3.5 -7.0 -11.5 -17.0 -23.5 -31.0 -39.5 ...
  * Zl          (Zl) float64 0.0 -2.0 -5.0 -9.0 -14.0 -20.0 -27.0 -35.0 ...
  * Zp1         (Zp1) float64 0.0 -2.0 -5.0 -9.0 -14.0 -20.0 -27.0 -35.0 ...
  * Zu          (Zu) float64 -2.0 -5.0 -9.0 -14.0 -20.0 -27.0 -35.0 -44.0 ...
  * time        (time) datetime64[ns] 2007-09-01 2007-09-02 2007-09-03 ...
Data variables:
    drC         (Zp1) float64 -1.0 -2.5 -3.5 -4.5 -5.5 -6.5 -7.5 -8.5 -9.5 ...
    drF         (Z) float64 -2.0 -3.0 -4.0 -5.0 -6.0 -7.0 -8.0 -9.0 -10.0 ...
    

Vertical sections
-------------------
Extract vertical sections from the model similarly to ship surveys conducted by observational oceanographer. Then use OceanSpy to plot the vertical sections.  
NOTE: Surveys need heavy interpolations that are immediately computed by OceanSpy. Use a light survey to test this functionality.

In [10]:
ds_survey, info_survey = ospy.subsample.survey(ds,
                                               info,
                                               lat1       =  66.9,   
                                               lon1       = -29.8,
                                               lat2       =  65.5,
                                               lon2       = -24.6,
                                               delta_km   = 2,
                                               varList    = ['U', 'V', 'Depth'],
                                               depthRange = [0, -700],
                                               timeRange  = ['2008-02-01T00', '2008-02-01T18'],
                                               deep_copy  = True)
_ = ospy.visualize.interactive(ds_survey, info_survey)

regrid_method : bilinear
description : fluid thickness in r coordinates (at rest)

Dimension : dist_VS
      long_name : Distance from vertex 1
      units : km


:Curve   [dist_VS]   (Depth)

![alt](images/tutorial_survey.png)

### Rotate axis

Often times oceanographers are interested in the velocity components tangential and orthogonal to a section. The axis can be rotated using ``ospy.compute``:

In [11]:
ds_rot, info_rot = ospy.compute.ort_Vel(ds_survey, info_survey, deep_copy=True)
ds_rot, info_rot = ospy.compute.tan_Vel(ds_rot, info_rot)
_ = ospy.visualize.interactive(ds_rot.ort_Vel, info_rot, hvplot_kwargs={'kind': 'contourf',
                                                                        'cmap':'seismic', 
                                                                        'clim':(-1,1), 
                                                                        'levels':20})

units : m/s
long_name : orthogonal velocity
direction : positive: flow keeps larger distances to the right
rotation_angle : 328.68838107901024 deg (positive: counterclockwise)
history : Computed offline by OceanSpy

Dimension : time
      long_name : model_time
      axis : time
Dimension : Z
      units : meters
      long_name : vertical coordinate of cell center
      positive : up
      axis : Z
Dimension : dist_VS
      long_name : Distance from vertex 1
      units : km


:DynamicMap   [time]
   :Polygons   [dist_VS,Z]   (ort_Vel)

![alt](images/tutorial_rotation.png)